In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import copy
import sys
import datetime
import iris
from pyhdf.SD import SD, SDC
import time

os.environ['CIS_PLUGIN_HOME'] = '/home/users/pete_nut/plugins/'
from cis import read_data, read_data_list, get_variables


In [11]:
def delete_no_overlap(filelist):
    delfiles=[]
    for filename in filelist:
        #print(filename)
        dataset = SD(filename, SDC.READ)
        lats = dataset.select('Latitude').get()
        lons = dataset.select('Longitude').get()
        #print(lats.min(), lats.max(), lons.min(), lons.max())
        if (lats.max() < -35.  or lats.min() > 0 or lons.max() < -90 or lons.min() > -70 or lons.max()>170 ):
            #print(filename)
            delfiles.append(filename)
        else: print(filename)
    
    for filename in delfiles:
        filelist.remove(filename)
    return filelist



In [15]:
months=['00','01','02','03','04','05','06','07','08','09','10','11','12']
modis_month = months[2]
days=['01','02','03','04','05','06','07','08','09',
      '10','11','12','13','14','15','16','17','18','19',
      '20','21','22','23','24','25','26','27','28','29', '30','31']

modis_day = days[0]
modis_year = '2018'
modis_dir = '/neodc/modis/data/MYD06_L2/collection61/{}/{}/{}/'

#ROI is UTC-5 so there is no daylight before ~1200

modis_daily = sorted(glob.glob(modis_dir.format(modis_year, modis_month, modis_day) + '*.hdf'))
end_day = glob.glob(modis_dir.format(modis_year, modis_month, modis_day) + '*.2355.*.hdf')[0]
end_day = modis_daily.index(end_day)
start_time=glob.glob(modis_dir.format(modis_year, modis_month, modis_day) + '*.1500.*.hdf')[0]
start_time=modis_daily.index(start_time)
modis_daily = modis_daily[start_time:end_day]

In [16]:
modis_daily=delete_no_overlap(modis_daily)

/neodc/modis/data/MYD06_L2/collection61/2018/02/01/MYD06_L2.A2018032.1840.061.2018033162742.hdf
/neodc/modis/data/MYD06_L2/collection61/2018/02/01/MYD06_L2.A2018032.1845.061.2018033163753.hdf
/neodc/modis/data/MYD06_L2/collection61/2018/02/01/MYD06_L2.A2018032.1850.061.2018033163352.hdf
/neodc/modis/data/MYD06_L2/collection61/2018/02/01/MYD06_L2.A2018032.2020.061.2018033162809.hdf
/neodc/modis/data/MYD06_L2/collection61/2018/02/01/MYD06_L2.A2018032.2025.061.2018033163225.hdf


In [19]:
mod_hr=read_data_list(modis_daily,
                   ['Cloud_Fraction',
                    'Cloud_Top_Temperature'])
mod_lr=read_data_list(modis_daily,
                      ['Cloud_Effective_Radius','Cloud_Water_Path'], product='MOD06_HACK')

In [20]:
albedo=read_data_list(modis_daily, 'Single_Scatter_Albedo_Liq')

In [25]:
albedo

[<cis 'UngriddedData' of Ungridded data: Liquid Water single scatter albedo from the phase functions used  to generate the forward lookup tables / (none) 
>]

In [ ]:
agg_mod_lr=mod_lr.aggregate(x=[-90.0,-70.,0.1], y=[-35.0,0.0,0.1])
#agg_mod_hr=mod_hr.aggregate(x=[-90.0,-70.,0.1], y=[-35.0,0.0,0.1])

In [ ]:
from cis.data_io.gridded_data import GriddedDataList
all_data=GriddedDataList([agg_mod_hr[0],agg_mod_hr[3], agg_mod_lr[0], agg_mod_lr[3],agg_mod_lr[6]])
all_data.save_data('/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/colloc_emission_modis/'+ modis_month + modis_day+".nc")